<a href="https://colab.research.google.com/github/Forkou-francine/tp_atl_big_data/blob/main/Tp_notation_films.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TP: PySpark pour analyse de données de films.**

**Objectif**: Utiliser PySpark pour manipuler un jeu de données sur les films, effectuer des transformations simples et réaliser des analyses

### **Travail réalisé par: Ange-Francine PENE FORKOU**


###**Étape 1: Prise en main - Préparation des données**
1. Charger le fichier CSV contenant les données des films dans un DataFrame PySpark :

In [7]:
#Import des librairies
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, explode, split, avg

#Instanciation d'une session spark
spark = SparkSession.builder.appName('RottenTomatoesMovieAnalysis').getOrCreate()

In [23]:
# Chargement du fichier CSV
file_path = "Rotten_Tomatoes_Movies.csv"

#Création du dataFrame qui contiendra les données des films
df_movies = spark.read.csv(file_path, header=True, inferSchema=True)

# Afficher un aperçu des données
print(f"Nombre de films avant nettoyage : {df_movies.count()}")
# df_movies.show(5)

Nombre de films avant nettoyage : 16638


2.   Nettoyer les données en supprimant les lignes vides ou invalides :


*   Supprimer les lignes avec des valeurs nulles ou invalides

In [9]:
# Liste des colonnes importantes à conserver pour l'analyse
"""
  Les colonnes sont sélectionnées car elles contiennent des informations
  critiques pour l'analyse comme les dates, les notes, et les noms des studios/réalisateurs.
"""
list_columns = ["tomatometer_rating", "critics_consensus", "in_theaters_date", "on_streaming_date", "studio_name", "directors"]

# Suppression des lignes contenant des valeurs nulles dans les colonnes importantes
df_movies_cleaned = df_movies.dropna(subset=list_columns)
#df_movies_cleaned.show(10)



3. Convertir les colonnes de dates au format "yyyy-mm-dd"


In [25]:
# Conversion de la colonne "in_theaters_date"
df_movies_cleaned = df_movies_cleaned.withColumn(
    "in_theaters_date", to_date(col("in_theaters_date"), "yyyy-MM-dd")
)

# Conversion de la colonne "on_streaming_date"
df_movies_cleaned = df_movies_cleaned.withColumn(
    "on_streaming_date", to_date(col("on_streaming_date"), "yyyy-MM-dd")
)
print(f"Nombre de films après nettoyage : {df_movies_cleaned.count()}")

# Vérifier si des colonnes importantes contiennent encore des valeurs nulles
null_counts = df_movies_cleaned.select([col(c).isNull().cast("int").alias(c) for c in list_columns]).groupBy().sum()
null_counts.show()


Nombre de films après nettoyage : 9085
+-----------------------+----------------------+---------------------+----------------------+----------------+--------------+
|sum(tomatometer_rating)|sum(critics_consensus)|sum(in_theaters_date)|sum(on_streaming_date)|sum(studio_name)|sum(directors)|
+-----------------------+----------------------+---------------------+----------------------+----------------+--------------+
|                      0|                     0|                 3174|                  2819|               0|             0|
+-----------------------+----------------------+---------------------+----------------------+----------------+--------------+



### **Etape 2: Manipulation et analyse des données**
1. Filtrage

In [10]:
# Les films avec une note très basse (tomatometer_rating < 20)

""" Ces films peuvent être utilisés pour analyser les tendances des échecs critiques."""
df_movies_low_rating = df_movies_cleaned.filter(col("tomatometer_rating") < 20)
df_movies_low_rating.show(5)

In [11]:
# Les films sortis (en cinema) après l'année 2000

""" Cela permet de se concentrer sur les films récents et pertinents pour analyser
les nouvelles tendances."""
df_movies_after_2000 = df_movies_cleaned.filter(col("in_theaters_date") > "2000-01-01")
df_movies_after_2000.show(5)

2. Moyennes

In [20]:
# La note moyenne des films par studio

""" Cela permet d'identifier les studios avec les meilleures (ou les pires)
performances en terme de critiques."""
df_avg_rating_studio = df_movies_cleaned.groupBy("studio_name").agg(avg("tomatometer_rating").alias("avg_rating"))
df_avg_rating_studio.show(5)

+--------------------+-----------------+
|         studio_name|       avg_rating|
+--------------------+-----------------+
|    Relativity Media|33.32142857142857|
|  New World Pictures|69.66666666666667|
| and seeks out th...|             NULL|
|       Nicholas Hope|             NULL|
|Alluvial Film Com...|             92.0|
+--------------------+-----------------+
only showing top 5 rows



In [13]:
# La moyenne des films par directeurs
df_avg_rating_director = df_movies_cleaned.groupBy("directors").agg(avg("tomatometer_rating").alias("avg_rating"))
df_avg_rating_director.show(5)

+--------------------+----------+
|           directors|avg_rating|
+--------------------+----------+
|Comedy, Science F...|      NULL|
|    Laurence Olivier|      91.0|
|        Jim Jarmusch|    74.625|
| more established...|      NULL|
| ""Amreeka"" is a...|      NULL|
+--------------------+----------+
only showing top 5 rows



### **Etape 3 : Utilisation de fonctions avancées**



1.   Diviser les genres multiples d'une colone en genres individuels



In [28]:
# On divise les genres selon le séparateur (,)
df_individual_genres = df_movies_cleaned.withColumn("genre", explode(split(col("genre"), ", ")))
df_individual_genres.show(5)

# On peut également compter les occurrences de chaque genre
genre_counts = df_individual_genres.groupBy("genre").count().orderBy(col("count").desc())
genre_counts.show()

+--------------------+--------------------+--------------------+------+--------------------+-----------------+-----------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|         movie_title|          movie_info|   critics_consensus|rating|               genre|        directors|          writers|                cast|in_theaters_date|on_streaming_date|runtime_in_minutes|         studio_name|tomatometer_status|tomatometer_rating|tomatometer_count|audience_rating|audience_count|
+--------------------+--------------------+--------------------+------+--------------------+-----------------+-----------------+--------------------+----------------+-----------------+------------------+--------------------+------------------+------------------+-----------------+---------------+--------------+
|Percy Jackson & t...|A teenager discov...|Though it may see...|

2. Calculer la durée moyennes des films pour chaque genre

In [17]:
df_avg_duration_genre = df_movies_cleaned.groupBy("genre").agg(avg("runtime_in_minutes").alias("avg_duration"))
df_avg_duration_genre.show(15)

+--------------------+------------------+
|               genre|      avg_duration|
+--------------------+------------------+
|         and Scamper|              NULL|
|Comedy, Science F...|              96.9|
|Comedy, Drama, Te...|             102.5|
| made partly out ...|              NULL|
| the Michigan Mil...|              NULL|
| take on new lust...|              NULL|
| and a beautiful ...|              NULL|
|Action & Adventur...|103.66666666666667|
| foolhardy docume...|              NULL|
|Action & Adventur...|103.76666666666667|
| Simon and Theodo...|              NULL|
|Animation, Art Ho...|              65.0|
|   Drama, Television|             112.2|
|Drama, Western, R...|137.66666666666666|
|Da Sweet Blood of...|              NULL|
+--------------------+------------------+
only showing top 15 rows



On peut également calculer la corrélation pour observer si les films avec une durée plus longue obtiennent de meilleures notes

In [30]:
df_correlation = df_movies_cleaned.select(
    col("runtime_in_minutes").cast("double"),
    col("tomatometer_rating").cast("double")
)

df_correlation.corr("runtime_in_minutes", "tomatometer_rating")


0.6968612216450497

On obtient une corrélation de 0.6968612216450497 entre "runtime_in_minutes" (la durée du film) et "tomatometer_rating" (note du critique critique)qui est modérément forte et positive. Cela signifie que lorsque la durée du film augmente, la note critique tend à augmenter aussi, mais ce n'est pas une règle absolue.